In [ ]:
%cd ..
%reload_ext autoreload
%autoreload 2

In [ ]:
import eos
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pyrender
import trimesh
from face_reconstruction.model import BaselFaceModel

from env import BASEL_FACE_MODEL_PATH

# 1. Load the face model
The Basel Face Model comes in 3 different versions:
 - Face mask without ears `face12.h5`
 - Face mask with ears `bfm.h5`
 - Full head model `fullHead.h5`

In [ ]:
h5 = h5py.File(f"{BASEL_FACE_MODEL_PATH}/model2019_face12.h5")
#h5 = h5py.File(f"{BASEL_FACE_MODEL_PATH}/model2019_bfm.h5")
#h5 = h5py.File(f"{BASEL_FACE_MODEL_PATH}/model2019_fullHead.h5")

In [ ]:
def print_keys_rec(node, level=0):
    if hasattr(node, 'keys'):
        for k in node.keys():
            if hasattr(node[k], 'keys'):
                print(''.join(['  ']*level) + '- ' + k)
                print_keys_rec(node[k], level + 1)
            else:
                print(''.join(['  ']*level) + '- ' + k + ': ' + str(node[k]))

## 1.1. Investigate the file structure

In [ ]:
print_keys_rec(h5)

# 2. Build the model
It is comprised of 3 parts: A shape model, a color model and an expression model

In [ ]:
print(f"Number of vertices in face model: ", h5['shape/model/mean'][()].shape[0] / 3) # Is the vertex information divided into groups of 3 (x, y, z)?

In [ ]:
bfm = BaselFaceModel.from_h5("model2019_face12.h5")

If you get a `MemoryError: bad allocation`, restart the notebook

# 3. Sample a 3D face from the model using some coefficients

In [ ]:
n_shape_coefficients = bfm.get_n_shape_coefficients()
n_expression_coefficients = bfm.get_n_expression_coefficients()
n_color_coefficients = bfm.get_n_color_coefficients()

In [ ]:
face_mesh = bfm.draw_sample(
    shape_coefficients=[0.9 for _ in range(n_shape_coefficients)], 
    expression_coefficients=[0.1 for _ in range(n_expression_coefficients)], 
    color_coefficients=[0.9 for _ in range(n_color_coefficients)])

In [ ]:
face_trimesh = bfm.convert_to_trimesh(face_mesh)

# 4. Render the 3D face interactively

To get Pyrender running in Windows: 
Change in `pyrender/platforms/pyglet_platform.py`
```{python}
def make_uncurrent(self):
    try:
        import pyglet.gl.xlib
        pyglet.gl.xlib.glx.glXMakeContextCurrent(self._window.context.x_display, 0, 0, None)
    except:
        pass
```

In [ ]:
perspective_camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.414)
directional_light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=2.0)

In [ ]:
scene = pyrender.Scene()
scene.add(pyrender.Mesh.from_trimesh(face_trimesh))
scene.add(perspective_camera, pose=np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 300], [0, 0, 0, 1]])) # Position camera just in front of face
scene.add(directional_light)

In [ ]:
pyrender.Viewer(scene, use_raymond_lighting=True)

# 5. Render random 3D faces into images

In [ ]:
for _ in range(10):
    r = pyrender.OffscreenRenderer(1200, 800)
    scene = pyrender.Scene()

    scene.add(perspective_camera, pose=np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 300], [0, 0, 0, 1]]))  # Position camera just in front of face
    scene.add(directional_light)
    face_mesh = bfm.draw_sample(
        shape_coefficients=np.random.uniform(low=-2, high=2, size=n_shape_coefficients),
        expression_coefficients=np.random.uniform(low=-2, high=2, size=n_expression_coefficients),
        color_coefficients=np.random.uniform(low=-2, high=2, size=n_color_coefficients), )
    face_trimesh = trimesh.Trimesh(vertices=face_mesh.vertices, faces=face_mesh.tvi, vertex_colors=face_mesh.colors, face_colors=face_mesh.colors)
    scene.add(pyrender.Mesh.from_trimesh(face_trimesh))

    color, depth = r.render(scene)
    plt.figure(figsize=(12, 8))
    plt.imshow(color)
    plt.show()
    r.delete()